# Retreive Experimental Formation Energies from MP Thermo database

This notebook assembles `MP_expt_energy_all.json`, which contains all of the experimental formation energies in the MP database

---
## Header

#### Global variables

In [5]:
PROJECT = 'SCAN project'

#### Imports

In [6]:
import os
import re
import numpy as np
import xlrd
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

In [7]:
from pymatgen.ext.matproj import MPRester, MPRestError
from monty.serialization import loadfn, dumpfn
from pymatgen import Structure, Composition
from pymatgen.analysis.reaction_calculator import ComputedEntry, ComputedReaction
from pymatgen.util.plotting import pretty_plot, periodic_table_heatmap

from scipy.stats import linregress
from adjustText import adjust_text
from sklearn.metrics import max_error, mean_absolute_error, mean_squared_error

#### Settings and utility functions

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Set Working Directory

In [9]:
workdir = re.sub("(?<={})[\w\W]*".format(PROJECT), "", os.getcwd())
os.chdir(workdir)

data_dir = os.path.join('2_raw data')
pipeline_dir = os.path.join('3_data analysis','2_pipeline')
output_dir = os.path.join('3_data analysis','3_output')

---
## Main Code

### List of all the formulas present in the MPThermo experimental database

In [10]:
mpthermo_cpds = ['Ag', 'AgBr', 'AgBrO3', 'AgCl', 'AgF', 'AgI', 'Ag2O', 'Ag2CO3', 'Ag2CrO4', 'AgNO3', 'Ag2S', 'Ag2SO4', 'Ag2Se', 'Al', 'AlAs', 'AlBr', 'AlBr3', 'Al2Br6', 'Ag2Te', 'Ag2WO4', 'Al4C3', 'AlCl', 'AlCl2', 'AlCl3', 'Al2Cl6', 'AlF', 'AlF2', 'AlF3', 'Al2F6', 'AlH3', 'AlI3', 'Al2I6', 'AlN', 'Al2O3', 'Al2O', 'Al4B2O9', 'AlOCl', 'AlOOH', 'AlOF', 'AlP', 'AlPO4', 'AlS', 'Al2S3', 'Al2(SO4)3', 'AlSb', 'AlSe', 'Al2Se3', 'Al6Si2O13', 'Al2SiO5', 'Al2Te3', 'Al2TiO5', 'As', 'As2', 'As4', 'AsCl3', 'AsF3', 'AsH3', 'AsI3', 'As2O3', 'AsBr3', 'As2O5', 'As2S3', 'AsS', 'As2Se3', 'As2Te3', 'Au', 'AuBr', 'AuCl', 'AuCl3', 'AuH', 'AuI', 'Au2O3', 'Au2P3', 'AuPb2', 'AuS', 'AuSb2', 'AuSe', 'AuSn', 'AuTe2', 'B', 'BBr3', 'B4C', 'BCl3', 'BF3', 'BH', 'B2H6', 'BI3', 'BN', 'B2O3', 'BOCl', 'B(OH)2', 'B(OH)3', 'BP', 'B2S3', 'Ba', 'BaBr2', 'BaC2', 'BaCl2', 'BaF2', 'BaH2', 'BAl2', 'Ba3N2', 'BaO', 'BaO2', 'Ba3Al2O6', 'BaAl2O4', 'BaCO3', 'BaHfO3', 'BaMoO4', 'Ba(NO3)2', 'BaS', 'BaSO4', 'BaSiO3', 'Ba2SiO4', 'Ba2Sn', 'Ba2TiO4', 'BaTiO3', 'BaUO4', 'BaV2O6', 'BaZrO3', 'Be', 'BeBr2', 'Be2C', 'BeCl2', 'Be2Cl4', 'BeF', 'BeF2', 'BeH2', 'BeI2', 'Be3N2', 'BeO', 'BeAl2O4', 'BeAl6O10', 'Be3B2O6', 'Be(OH)2', 'BeS', 'BeSO4', 'Be2SiO4', 'BeWO4', 'Bi', 'Bi2', 'BiBr3', 'BiCl', 'BiCl3', 'BiF', 'BiF3', 'BiH', 'BiI3', 'Bi2O3', 'BiOCl', 'Bi2S3', 'Bi2(SO4)3', 'Bi2Se3', 'Bi2Te3', 'Br2', 'Br', 'C', 'C2', 'C3', 'CBr4', 'CCl4', 'CF2', 'CF3', 'CF4', 'C2F6', 'CF2Cl2', 'CFCl3', 'CH2', 'CH3', 'CH4', 'C2H2', 'C2H4', 'C2H6', 'CH3Cl', 'CN', 'C2N2', 'CO', 'CO2', 'COCl2', 'COF', 'CH2O', 'CH3OH', 'C2H5OH', 'CS', 'CS2', 'COS', 'Ca', 'CaAl2', 'CaBr2', 'COF2', 'CaC2', 'CaCl2', 'CaF2', 'CaH', 'CaH2', 'CAl2', 'Ca3N2', 'CaO', 'Ca3Al2O6', 'CaAl2O4', 'CaAl4O7', 'Ca3B2O6', 'Ca2B2O5', 'CaB2O4', 'CaB4O7', 'CaMg2', 'CaCO3', 'CaOCl2', 'CaCr2O4', 'Ca2Fe2O5', 'CaFe2O4', 'CaGeO3', 'CaOH', 'Ca(OH)2', 'CaHfO3', 'CaMg(CO3)2', 'CaMoO4', 'Ca(NO3)2', 'CaNb2O6', 'Ca3P2', 'Ca3(PO4)2', 'Ca2P2O7', 'CaHPO4', 'Ca2Pb', 'CaS', 'CaSO4', 'Ca3Sb2', 'CaSe', 'CaSi', 'Ca3SiO5', 'Ca2SiO4', 'Ca3Si2O7', 'CaSiO3', 'Ca4Ti3O10', 'CaTiO3', 'CaUO4', 'Ca3V2O8', 'Ca2V2O7', 'CaV2O6', 'CaWO4', 'Ca3WO6', 'CaZrO3', 'Cd', 'Cd3As2', 'CdBr2', 'CdCl2', 'CdF2', 'CdI2', 'CdO', 'CdAl2O4', 'CdCO3', 'CdGa2O4', 'Cd(NO3)2', 'CdS', 'CdSb', 'CdSe', 'CdSeO3', 'CdSiO3', 'CdTe', 'CdTiO3', 'Ce', 'CeBr3', 'Ce2C3', 'CeC2', 'CdSO4', 'CeCl3', 'CeF3', 'CeH2', 'CeI3', 'CeN', 'Ce2O3', 'CeO2', 'CeAlO3', 'CeCrO3', 'CeS', 'Ce3S4', 'Ce2S3', 'Ce2O2S', 'Ce2(SO4)3', 'Cl2', 'Cl', 'ClF', 'ClF3', 'Cl2O', 'ClO2', 'Co', 'CoAl', 'CoBr2', 'CoCl2', 'CoF2', 'CoF3', 'CoI2', 'CoO', 'Co3O4', 'CoAl2O4', 'CoCO3', 'CoCr2O4', 'CoFe2O4', 'Co(OH)2', 'Co2P', 'CoP', 'CoP3', 'Co3S4', 'CoS2', 'CoSO4', 'CoSb', 'CoSeO3', 'CoSi', 'CoSi2', 'Co2SiO4', 'CoTiO3', 'CoWO4', 'Cr', 'Cr5B3', 'CrB', 'Cr3B4', 'CrB2', 'CrBr2', 'Cr23C6', 'Cr7C3', 'Cr3C2', 'CrCl2', 'CrCl3', 'CrF2', 'CrF3', 'CrI2', 'Cr13', 'Cr2N', 'CrN', 'Cr2O3', 'CrO3', 'Cr(CO)6', 'CrO2Cl2', 'CrS', 'Cr2(SO4)3', 'Cr3Si', 'Cr5Si3', 'CrSi', 'CrSi2', 'Cs', 'Cs2', 'CsBr', 'CsCl', 'CsF', 'CsH', 'CsI', 'Cs2O', 'CsO2', 'CsBO2', 'Cs2CO3', 'CsClO4', 'Cs2CrO4', 'CsOH', 'Cs2MoO4', 'CsNO2', 'CsNO3', 'Cs2SO4', 'Cs2SiO3', 'Cs2UO4', 'Cu', 'CuBr', 'CuCl', 'CuCl2', 'CuF2', 'CuI', 'CuCN', 'Cu2O', 'CuO', 'CuAl2O4', 'CuCO3', 'CuCr2O4', 'CuFeO2', 'CuFe2O4', 'Cu(OH)2', 'Cu3P', 'Cu2S', 'CuS', 'Cu5FeS4', 'CuFeS2', 'CuSO4', 'Cu2Sb', 'Cu2Se', 'CuSe', 'Cu2Te', 'Dy', 'DyCl3', 'DyF3', 'Dy2O3', 'Er', 'ErCl3', 'ErF3', 'Er2O3', 'Eu', 'EuCl3', 'EuF3', 'EuN', 'EuO', 'Eu2O3', 'EuS', 'F2', 'F', 'F2O', 'Fe', 'Fe2B', 'FeB', 'FeBr2', 'Fe3C', 'FeCl2', 'Fe2Cl4', 'FeCl3', 'Fe2Cl6', 'FeF2', 'Fe4N', 'Fe2O3', 'Fe3O4', 'FeAl2O4', 'FeCO3', 'Fe(CO)5', 'FeOCl', 'FeCr2O4', 'FeOOH', 'Fe3Mo2', 'FeMoO4', 'Fe3P', 'Fe2P', 'FeS', 'FeS2', 'FeSO4', 'Fe2(SO4)3', 'Fe2(SeO3)3', 'FeSi', 'Fe2SiO4', 'FeTi', 'Fe2Ti', 'Fe2TiO4', 'FeTiO3', 'FeWO4', 'Ga', 'GaAs', 'GaBr3', 'GaCl', 'GaCl3', 'GaI3', 'GaN', 'Ga2O3', 'GaP', 'Ga2S', 'GaS', 'Ga2S3', 'GaSb', 'GaSe', 'Ga2Se3', 'GaTe', 'Ga2Te3', 'Gd', 'GdBr3', 'GdCl3', 'GdF3', 'GdI3', 'Gd2O3', 'GdOCl', 'Ge', 'GeBr4', 'GeCl4', 'GeF4', 'GeH4', 'GeI4', 'GeO', 'GeO2', 'GeP', 'GeS', 'GeS2', 'GeSe', 'GeSe2', 'Co(NO3)2', 'GeTe', 'H', 'H2', 'HBr', 'HCl', 'HF', 'HI', 'HCN', 'H2O', 'H2O2', 'HBO2', 'H3BO3', 'HOF', 'HNO3', 'HCNO', 'H3PO4', 'H2S', 'H2S2', 'H2SO4', 'H2Se', 'H2Te', 'Hf', 'HfB2', 'HfBr4', 'HfC', 'HfCl4', 'HfF4', 'HfI4', 'HfN', 'HfO2', 'Hg', 'HgBr', 'HgBr2', 'HgCl', 'HgCl2', 'HgF', 'HgF2', 'HgI', 'HgI2', 'HgO', 'HgS', 'Hg2SO4', 'HgSO4', 'HgSe', 'HgSeO3', 'HgTe', 'Ho', 'HoCl3', 'HoF3', 'Ho2O3', 'I2', 'I', 'In', 'InAs', 'InBr', 'InBr3', 'InCl', 'InCl2', 'InCl3', 'InI', 'InI3', 'InN', 'In2O3', 'InP', 'InS', 'In2S3', 'In2(SO4)3', 'InSb', 'InSe', 'In2Se3', 'In2Te', 'InTe', 'In2Te3', 'Ir', 'IrCl3', 'IrF6', 'IrO2', 'Ir2S3', 'IrS2', 'K', 'K2', 'KBr', 'KCl', 'KCaCl3', 'KF', 'K3AlF6', 'KBF4', 'KH', 'KHF2', 'KI', 'KCN', 'K2O', 'K2O2', 'KO2', 'KBO2', 'K2B4O7', 'K2CO3', 'KClO3', 'KClO4', 'K2CrO4', 'KFeO2', 'KOH', 'KNO3', 'K3PO4', 'K2HPO4', 'KH2PO4', 'K2S', 'K2SO4', 'K2Si2O5', 'K2SiO3', 'K2Si4O9', 'KAl(SO4)2', 'K2WO4', 'La', 'LaAl2', 'LaBr3', 'LaCl3', 'LaF3', 'LaH2', 'LaI3', 'LaN', 'La2O3', 'LaAlO3', 'LaOCl', 'La3(MoO4)3', 'LaPO4', 'La2S3', 'LaS', 'La2Se3', 'LaSe', 'La2Te3', 'Li', 'Li2', 'LiBr', 'Li2C2', 'LiCl', 'LiF', 'Li2F2', 'Li3F3', 'Li3AlF6', 'Li2BeF4', 'LiBeF3', 'LiH', 'LiI', 'Li3N', 'Li2O', 'Li2O2', 'LiAlO2', 'Li3AsO4', 'LiBO2', 'Li2B4O7', 'LiB3O5', 'Li2B8O13', 'Li2CO3', 'LiClO4', 'LiFeO2', 'LiOH', 'Li2HfO3', 'LiNO3', 'LiNbO3', 'LiPO3', 'Li2S', 'Li2SO4', 'Li2Se', 'Li2SiO3', 'LiTaO3', 'Li2Te', 'Li2TiO3', 'Li2WO4', 'Li2ZrO3', 'Lu', 'LuF3', 'Lu2O3', 'Mg', 'MgB2', 'MgB4', 'MgB12', 'MgBr2', 'MgCe', 'MgCl2', 'Mg2Cu', 'MgCu2', 'MgF2', 'Mg2Ge', 'MgH2', 'MgI2', 'Mg3N2', 'MgO', 'MgAl2O4', 'MgCO3', 'MgCr2O4', 'MgMoO4', 'Mg(NO3)2', 'Mg2Pb', 'MgS', 'MgSO4', 'Mg3Sb2', 'Mg2Si', 'Mg2SiO4', 'MgSiO3', 'MgTe', 'Mg2TiO4', 'MgTiO3', 'MgTi2O5', 'MgUO4', 'MgV2O6', 'Mg2V2O7', 'MgWO4', 'Mn', 'MgGeO3', 'Mg(OH)2', 'Mg3(PO4)2', 'MnAs', 'Mn2B', 'MnB', 'Mn3B4', 'MnB2', 'MnBr2', 'MnCl2', 'MnF2', 'MnO', 'Mn3O4', 'Mn2O3', 'MnO2', 'MnAl2O4', 'MnCO3', 'MnFe2O4', 'MnMoO4', 'MnP', 'MnS', 'MnS2', 'MnSO4', 'Mn2Sb', 'MnSb', 'MnSe', 'Mn3Si', 'Mn5Si3', 'MnSi', 'Mn2SiO4', 'MnSiO3', 'MnSn2', 'MnTe', 'MnTe2', 'Mn2TiO4', 'MnTiO3', 'MnWO4', 'Mo', 'Mo2B', 'MoB', 'Mo2C', 'MoCl4', 'MoCl5', 'MoC15', 'MoCl6', 'MoF5', 'Mo2F10', 'MoF6', 'MoO2', 'MoO3', 'Mo(CO)6', 'Mo2S3', 'MoS2', 'Mo3Si', 'Mo5Si3', 'MoSi2', 'N2', 'N', 'NH3', 'NH4Br', 'NH4Cl', 'NH4F', 'NH4I', 'NO', 'N2O3', 'NO2', 'N2O4', 'N2O5', 'NO3', 'NOBr', 'NOCl', 'NH4ClO4', 'N2O', 'NOF', 'NH4NO3', '(NH4)2SO4', 'Na', 'Na2', 'NaBr', 'NaCl', 'NaAlCl4', 'NaF', 'Na3AlF6', 'NaBF4', 'NaH', 'NaI', 'NaCN', 'Na2O', 'Na2O2', 'NaO2', 'NaO', 'NaAlO2', 'Na3AsO4', 'NaBO2', 'Na2B4O7', 'NaB3O5', 'Na2B8O13', 'Na2CO3', 'NaClO4', 'NaClO3', 'NaCrO2', 'Na2CrO4', 'NaFeO2', 'NaOH', 'NaHCO3', 'Na2MoO4', 'Na2Mo2O7', 'NaNO3', 'NaPO3', 'Na3PO4', 'Na4P2O7', 'Na2S', 'Na2SO4', 'Na2SO3', 'Na2SiF6', 'Na4SiO4', 'Na2SiO3', 'Na2Si2O5', 'Na2Te', 'Na2TiO3', 'Na2Ti2O5', 'Na2Ti3O7', 'Na3UO4', 'Na2UO4', 'Na3VO4', 'Na4V2O7', 'NaVO3', 'Na2WO4', 'Nb', 'NbB2', 'NbBr5', 'Nb2C', 'NbC', 'NbCl4', 'NbCl5', 'NbCr2', 'NbF5', 'Nb2N', 'NbN', 'NbO', 'NbO2', 'Nb2O5', 'NbOCl2', 'NbOCl3', 'Nb5Si3', 'NbSi2', 'Nd', 'NdCl3', 'NdF3', 'NdH2', 'NdI3', 'Nd2O3', 'NdOCl', 'NdS', 'Nd2S3', 'Ni', 'Ni3Al', 'NiAl', 'Ni2Al3', 'NiAl3', 'NiAs', 'Ni3B', 'Ni2B', 'Ni4B3', 'NiB', 'NiBi', 'NiBr2', 'NiBr', 'NiCl2', 'NiCl', 'NiAl2Cl8', 'NiF2', 'NiF', 'NiH', 'NiI2', 'NiI', 'NiO', 'NiAl2O4', 'NiCO3', 'Ni(CO)4', 'NiCr2O4', 'Ni(OH)2', 'Ni3P', 'Ni5P2', 'Ni2P', 'Ni6P5', 'Ni3S2', 'NiS', 'Ni3S4', 'NiSO4', 'NiSb', 'NiSe2', 'NiSeO3', 'NiSi', 'Ni2SiO4', 'NiTe', 'Ni3Ti', 'NiTi2', 'NiTiO3', 'NiWO4', 'O2', 'O3', 'O', 'Os', 'OsO2', 'OsO4', 'OsS2', 'OsSe2', 'P', 'PbTe', 'PbTiO3', 'PbWO4', 'Pd', 'PdCl2', 'PdI2', 'PdO', 'Pd4S', 'PdS', 'PdS2', 'PdTe', 'PdTe2', 'Pr', 'PrCl3', 'PrF3', 'PrH2', 'PrI3', 'Pr2O3', 'P2', 'P4', 'PBr3', 'PCl3', 'PCl5', 'PF3', 'PF5', 'PH3', 'PN', 'P4O6', 'PrS', 'Pt', 'PtBr4', 'PtCl4', 'PtO2', 'PtS', 'PtS2', 'Pt5Se4', 'PuN', 'Rb', 'Rb2', 'RbBr', 'RbCl', 'RbF', 'RbH', 'RbI', 'Rb2O', 'RbBO2', 'Rb2CO3', 'RbOH', 'Rb2S', 'P2O5', 'P4O10', 'POBr3', 'POCl3', 'P4S3', 'PS', 'P2S3', 'Pb', 'PbBr2', 'Rb2SO4', 'Re', 'ReBr3', 'ReCl3', 'ReF6', 'ReO2', 'ReO3', 'Re2O7', 'ReS2', 'Re2S7', 'Re5Si3', 'ReSi', 'ReSi2', 'Rh', 'RhCl3', 'Rh2O3', 'RhO', 'Rh3S4', 'Rh2S3', 'Ru', 'RuCl3', 'PbCl2', 'PbF2', 'PbI2', 'PbO', 'Pb3O4', 'PbO2', 'PbCO3', 'RuCl4', 'RuF5', 'RuO2', 'RuO4', 'RuS2', 'S', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S2Cl2', 'SCl2', 'SF5', 'PbMoO4', 'Pb(NO3)2', 'PbS', 'PbSO4', 'PbSe', 'PbSeO3', 'Pb2SiO4', 'PbSiO3', 'SF6', 'SO', 'SO2', 'SO3', 'SOCl2', 'SO2Cl2', 'SO2F2', 'Sb', 'Sb2', 'Sb4', 'SbBr3', 'SbCl3', 'SbCl5', 'SbF3', 'SbH3', 'SbI3', 'Sb2O3', 'Sb2O5', 'SbOC1', 'Sb2S3', 'Sb2Se3', 'Sb2Te3', 'Sc', 'ScCl3', 'ScF3', 'ScN', 'Sc2O3', 'Se', 'Se2', 'Se3', 'Se4', 'Se5', 'Se6', 'Se2Cl2', 'SeCl2', 'SeCl4', 'SeF4', 'SeF5', 'SeF6', 'SeO', 'SeO2', 'Si', 'Si2', 'Si3', 'SiBr2', 'SiBr4', 'SiC', 'SiCl2', 'SiCl4', 'SiF2', 'SiF4', 'SiH', 'SiH4', 'SiH2Cl2', 'SiI2', 'SiI4', 'Si3N4', 'SiO', 'SiO2', 'SiOF2', 'SiP', 'SiS', 'SiS2', 'SiSe', 'Sm', 'SmC2', 'SmCl2', 'SmCl3', 'Sm2O3', 'SmOCl', 'SmS', 'SmSe', 'Sn', 'SnBr2', 'SnBr4', 'SnCl2', 'SnCl4', 'SnH4', 'SnI2', 'SnI4', 'SnO', 'SbO2', 'SnO2', 'Sn(NO3)2', 'SnS', 'Sn3S4', 'Sn2S3', 'SnS2', 'SnSO4', 'SnSe', 'SnSe2', 'SnTe', 'Sr', 'SrBr2', 'SrC2', 'SrCl2', 'SrF2', 'Si2H6', 'SrH2', 'SrI2', 'SrO', 'SrO2', 'SrAl2O4', 'SrB4O7', 'SrCO3', 'Sr(OH)2', 'SrHfO3', 'SrMoO4', 'SrS', 'SrSO4', 'Sr2SiO4', 'SrSiO3', 'Sr2TiO4', 'SrTiO3', 'SrWO4', 'SrZrO3', 'Ta', 'TaB2', 'TaBr3', 'Ta2C', 'TaC', 'TaCl3', 'TaCl4', 'TaCl5', 'TaCr2', 'TaFS', 'TaF5', 'TaFe2', 'Ta2N', 'TaN', 'TaO2', 'Ta2O5', 'TaS2', 'Ta5Si3', 'TaSi2', 'Tb', 'TbF3', 'TbCl3', 'Tb2O3', 'TbO2', 'Te', 'Te2', 'TeCl4', 'TeF4', 'TeF6', 'TeO', 'TeO2', 'Th', 'ThBr4', 'ThC', 'ThC2', 'ThCl4', 'ThF4', 'ThH2', 'ThI4', 'USe', 'U3Si', 'U3Si2', 'USi', 'USi2', 'USi3', 'V', 'VBr2', 'VBr3', 'VBr4', 'ThN', 'Th3N4', 'ThO', 'ThO2', 'ThOBr2', 'ThOCl2', 'ThOI2', 'Th2N2O', 'ThRe2', 'Th7Rh3', 'ThRh', 'VCl2', 'VCl3', 'VCl4', 'VF4', 'VF5', 'V3Ge', 'V5Ge3', 'VN', 'VO', 'V2O3', 'ThRh3', 'ThRh5', 'ThRu', 'Th7Ru3', 'ThS', 'Th2S3', 'ThS2', 'Th(SO4)2', 'Th3Si2', 'ThSi', 'VO2', 'V2O5', 'VOCl3', 'V3Si', 'V5Si3', 'VSi2', 'V3Sn', 'W', 'W2B', 'Th3Si5', 'ThSi2', 'Ti', 'TiB', 'TiB2', 'TiBr2', 'TiBr3', 'TiBr4', 'WB', 'W2C', 'WC', 'WCl4', 'WCl5', 'W2Cl10', 'WCl6', 'TiC', 'TiCl2', 'TiCl3', 'Ti2Cl6', 'TiCl4', 'TiCr2', 'TiF4', 'WF6', 'WO3', 'WO2', 'W(CO)6', 'WOCl4', 'WOF4', 'WO2I2', 'TiI4', 'TiN', 'TiO', 'Ti2O3', 'Ti3O5', 'TiO2', 'TiOCl', 'TiOCl2', 'TiOF', 'WS2', 'W5Si3', 'WSi2', 'Y', 'YCl3', 'YF3', 'YH2', 'YH3', 'TiOF2', 'TiS', 'TiS2', 'Ti5Si3', 'TiSi', 'TiSi2', 'Tl', 'TlBr', 'TlCl', 'Y13', 'YN', 'Y2O3', 'Yb', 'YbCl2', 'YbCl3', 'YbN', 'Yb2O3', 'Zn', 'TlF', 'TlI', 'Tl2O', 'Tl2O3', 'Tl2S', 'Tl2SO4', 'Tl2Se', 'TlSe', 'Tl2Te', 'Tm', 'Zn3As2', 'ZnBr2', 'ZnCl2', 'ZnF2', 'ZnI2', 'Zn3N2', 'ZnO', 'ZnAl2O4', 'ZnCO3', 'TmCl3', 'TmF3', 'Tm2O3', 'U', 'UAl2', 'UAl3', 'UAl4', 'UAs2', 'ZnCr2O4', 'ZnFe2O4', 'Zn(OH)2', 'Zn3P2', 'ZnP2', 'ZnS', 'ZnSO4', 'ZnSb', 'ZnSe', 'ZnSeO3', 'UB2', 'UB4', 'UB12', 'UBr4', 'UC', 'U2C3', 'UC2', 'UCl3', 'UCl4', 'Zn2SiO4', 'ZnTe', 'Zn2TiO4', 'ZnWO4', 'Zr', 'ZrB2', 'ZrBr4', 'ZrC', 'U2Cl8', 'UCl5', 'U2Cl10', 'UCl6', 'UF3', 'UF4', 'UF5', 'ZrCl4', 'ZrF4', 'ZrI4', 'ZrN', 'ZrO', 'ZrO2', 'ZrS2', 'ZrSiO4', 'AlFe', 'UF6', 'UFe2', 'UH3', 'UI4', 'UN', 'UO2', 'U4O9', 'U3O8', 'UO3', 'AgO', 'Ag2O3', 'BClO', 'MgO3Ti', 'Cl2Sr', 'UOBr2', 'UOBr3', 'UO2Br2', 'UOCl', 'UOCl2', 'U2O5Cl5', 'UOCl3', 'UO2Cl', '(UO2)2Cl3', 'UO2Cl2', 'UO2C12', 'I2Sr', 'UO2F2', 'UP', 'US', 'US2', 'US3', 'UO2SO4', 'AlHO2', 'Cs2O4S', 'FNO', 'Na2O4W', 'Li2O3Si', 'F2O2S', 'BLiO2', 'Cl2Mg', 'F4Ti', 'H2O4S', 'Mg2O4Ti', 'FLi', 'HLi', 'FHg', 'K2O4S', 'C6Cr23', 'O8U3', 'B4Mg', 'ClOTi', 'Cl3Fe', 'Cl2Fe', 'O4PbS', 'BeF3Li', 'Cl2S2', 'FeO4S', 'F2HK', 'AlLiO2', 'Br2Ca', 'F2Fe', 'Cl10W2', 'Cl5Ta', 'O2Si', 'BNaO2', 'O5Ti3', 'S2Si', 'FNa', 'HNa', 'INa', 'C2Li2', 'NV', 'OV', 'Cl2O2S', 'HSi', 'OSi', 'CHNO', 'Fe2O12S3', 'Cl3OP', 'O4SiZr', 'ClH4N', 'Br2Pb', 'Br4Zr', 'C2Cr3', 'CTa', 'BrH4N', 'B2Zr', 'FH', 'O3PbSi', 'F4Ge', 'Al6BeO10', 'CMgO3', 'F3Li3', 'B4Li2O7', 'CCl2F2', 'AlF6K3', 'Cl5Nb', 'C3Al4', 'B2Be3O6', 'CK2O3', 'Br2Ti', 'Br4Ti', 'HKO', 'Li2O3Ti', 'BKO2', 'Cl2Cu', 'AlF6Li3', 'F2OTi', 'Cl4OW', 'Cl2OTi', 'SSi', 'Br2Fe', 'F2Hg', 'O10P4', 'CSi', 'Be2O4Si', 'ILi', 'B2Ti', 'N4Si3', 'BHO2', 'Cl4Fe2', 'Cl6Fe2', 'F6W', 'B4Na2O7', 'Br3OP', 'O2Pb', 'F2Mg', 'Cl2Pb', 'C3Cr7', 'C4NiO4', 'CCl3F', 'CFO', 'O2Ti', 'CCl2O', 'FOTi', 'Na2O5Si2', 'Cl6Mo', 'Cl4Mo', 'Cl5Mo', 'Cl4Zr', 'O5V2', 'BrH', 'FHO', 'H3O4P', 'O3V2', 'CKN', 'ClCs', 'Al2BeO4', 'O4Pb3', 'F3P', 'F5P', 'Al2O5Si', 'B8Li2O13', 'B2Mg', 'H2O2Sr', 'H2Mg', 'C6MoO6', 'F4Si', 'F2Si', 'CaH2O2', 'BrK', 'ClHg', 'CHN', 'ClH', 'ClCu', 'CNa2O3', 'CCuN', 'Cl6W', 'Cl4W', 'Cl5W', 'Cl3P', 'Cl5P', 'OS', 'ClNi', 'Br5Nb', 'O2Zr', 'F6S', 'F5S', 'H4IN', 'I2Mg', 'CsHO', 'C5FeO5', 'BrHg', 'B4K2O7', 'F2OSi', 'ClH4NO4', 'Br2Mg', 'Na2O3Si', 'H2MgO2', 'O6P4', 'Cl2Ni', 'H4Si', 'Cl2H2Si', 'CF2O', 'Al6O13Si2', 'MgO4S', 'ClLi', 'Br3P', 'ClNO', 'NTi', 'H3P', 'ClLiO4', 'Cl2Hg', 'OTi', 'BTi', 'CTi', 'CLi2O3', 'Cl2Si', 'Cl4Si', 'F10Mo2', 'BeH2O2', 'CoO4S', 'I4Zr', 'BH3O3', 'O3Ti2', 'BeO4W', 'F4OW', 'CuH2O2', 'I4Si', 'I2Si', 'F2Pb', 'Mg3O8P2', 'MgO3Si', 'OSn', 'O4Pb2Si', 'AlClO', 'F6Mo', 'F5Mo', 'Br4Si', 'ClK', 'Cl2S', 'HNaO', 'CBe2', 'O3U', 'O2U', 'Al2MgO4', 'I2Pb', 'BrLi', 'BrNO', 'Br2Si', 'Br2Hg', 'AlFO', 'CZr', 'NZr', 'H3N', 'Na2O4S', 'AlF6Na3', 'MgO5Ti2', 'OZr', 'I4Ti', 'O3W', 'O2W', 'AlCl4Na', 'CuO4S', 'OZn', 'FK', 'CaHO', 'IK', 'HK', 'Cl3Ti', 'NP', 'OSr', 'SSr', 'ClNa', 'O2Sn', 'CNNa', 'O4SZn', 'BH2O2', 'Li2O4S', 'MgO4W', 'C6O6W', 'F4Zr', 'ClKO4', 'O3S', 'O2S', 'F2Sr', 'OPb', 'O2V', 'O2Th', 'HLiO', 'Cl2Co', 'Cl4Ti', 'O2Ta', 'Cl2Ti', 'ClNaO4', 'BeF4Li2', 'AlNaO2', 'Mg2O4Si', 'BrNa', 'Br3Ti', 'O5Ta2', 'CB4', 'BF4K', 'Br2Sr', 'K2O3Si', 'Hg2O4S', 'F2Li2']
len(mpthermo_cpds)

1639

In [19]:
with MPRester() as m:
    e=m.get_exp_entry('Li2O')

e.energy

-598.73

#### @Ayush - you'll need to change the directories and the reference to `mpthermo_cpds`

In [7]:
import os
if not os.path.isfile(os.path.join('3_data analysis','2_pipeline','MP_expt_energy_all.json')):
    MP_expt_energy = {}
    MP_expt_energy_un = {}
    with MPRester() as a:
        for i, k in tqdm(enumerate(mpthermo_cpds), total = len(mpthermo_cpds)):
            try:
                thermo = a.get_exp_entry(k)
                # convert the MPThermo energies from kJ/mol into eV/formula unit
                MP_expt_energy.update({k:thermo.energy/96.5})
            except:
                pass
    
    dumpfn(MP_expt_energy,os.path.join('3_data analysis','2_pipeline','MP_expt_energy_all.json'))
else:
    MP_expt_energy = loadfn(os.path.join('3_data analysis','2_pipeline','MP_expt_energy_all.json'))
    print('Loading data for {} compounds from disk'.format(len(MP_expt_energy)))

# Appendix
---

In [8]:
break

SyntaxError: 'break' outside loop (<ipython-input-8-6aaf1f276005>, line 1)